In [1]:
import matplotlib.pyplot as plt
import numpy as np
import idx2numpy


In [2]:
test_images = idx2numpy.convert_from_file('t10k-images.idx3-ubyte')
test_labels = idx2numpy.convert_from_file('t10k-labels.idx1-ubyte')
train_images = idx2numpy.convert_from_file('train-images.idx3-ubyte')
train_labels = idx2numpy.convert_from_file('train-labels.idx1-ubyte')

In [3]:
print(test_images.shape)
print(test_labels.shape)
print(train_images.shape)
print(train_labels.shape)


(10000, 28, 28)
(10000,)
(60000, 28, 28)
(60000,)


In [22]:
train_images_flat=train_images.reshape(train_images.shape[0], -1).T
test_images_flat=test_images.reshape(test_images.shape[0], -1).T

In [23]:
print(train_images_flat.shape)
print(test_images_flat.shape)
m = train_images_flat.shape[1]

(784, 60000)
(784, 10000)


In [24]:
W = np.random.randn(train_images_flat.shape[0], 1)*0.01  # W.shape is (n_x,1)
W = np.zeros((train_images_flat.shape[0], 1))
b = np.zeros((1,m)) #b.shape is (1,m)
X = train_images_flat/255

In [25]:
print(W.shape)
print(b.shape)

(784, 1)
(1, 60000)


In [26]:
def sigmoid(z):
    a = 1/(1+np.exp(-z))
    return a

In [27]:
def forward():
    global Z,A
    Z = np.dot(W.T, X) + b # Z.shape is (1,m)
    A = sigmoid(Z)
    Z.shape, A.shape

In [28]:
Y = train_labels.reshape(train_labels.shape[0],1).T # Y.shape is (1, m)
Y.shape

(1, 60000)

In [29]:
def cost():
    global L,J
# Calculate Loss
    L = -(Y*np.log(A) + (1-Y)*np.log(1-A)) # L.shape is (1,m)

    #calculate Cost
    J = np.sum(L, keepdims=True, axis=1)*(1/m) # J.shape is (1,1) if keepdims==True else J.shape is ()

In [30]:
# deivatives
def backward():
    global dZ, dW, db
    #dJ/dZ
    dZ = A-Y  # dZ.shape is (1,m)

    #dJ/dW = sum(dL/dW)/m  which should be equivalent to   np.dot(X,dZ.T)/m
    # (dW == np.dot(X, dZ.T)).all() :gives True

    dLdW = X*dZ  # dLdW.shape is (n_x, m)
    dW = np.sum(dLdW, keepdims=True, axis=1)/m  #dW.shape is (n_x,1) [same as W.shape]

    #dJ/db
    db = np.sum(dZ, keepdims=True, axis=1)/m  # db.shape = (1,1)

In [ ]:
#optimise

alpha = 0.009
iterations = 20

for i in range(iterations):
    forward()
    cost()
    print(f'Cost at {i}th loop = {J}')
    backward()
    W = W - alpha*dW
    b = b - alpha*db

Cost at 0th loop = [[140.68300016]]
Cost at 1th loop = [[317.26130545]]
Cost at 2th loop = [[535.05937543]]
Cost at 3th loop = [[765.06872665]]
Cost at 4th loop = [[998.85069766]]
Cost at 5th loop = [[1233.8969885]]
Cost at 6th loop = [[1469.3917869]]
Cost at 7th loop = [[1705.0525317]]
Cost at 8th loop = [[1940.77672743]]
Cost at 9th loop = [[2176.52580834]]
Cost at 10th loop = [[2412.28482678]]
Cost at 11th loop = [[2648.04785134]]
Cost at 12th loop = [[2883.81244606]]


<ipython-input-29-ed9116e788cb>:4: RuntimeWarning: divide by zero encountered in log
  L = -(Y*np.log(A) + (1-Y)*np.log(1-A)) # L.shape is (1,m)


Cost at 13th loop = [[inf]]
Cost at 14th loop = [[inf]]
Cost at 15th loop = [[inf]]
Cost at 16th loop = [[inf]]
Cost at 17th loop = [[inf]]


<ipython-input-29-ed9116e788cb>:4: RuntimeWarning: invalid value encountered in multiply
  L = -(Y*np.log(A) + (1-Y)*np.log(1-A)) # L.shape is (1,m)


Cost at 18th loop = [[nan]]


In [14]:
a=np.array([1,2,3])
b=np.array([1,2,3])

In [15]:
a*b

array([1, 4, 9])

In [16]:
np.multiply(a,b)

array([1, 4, 9])

In [17]:
def abc():
    global a,b,c
    a=5
    b=7
    c=10
    return

In [18]:
abc()


In [19]:
a


5

In [20]:
b

7